<a href="https://colab.research.google.com/github/zelalemamera-stonybrook/projects-sandbox/blob/main/Recurrent_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as T
!pip install torchcodec
import regex as re


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.7 MB/s eta 0:00:00


In [9]:
with open('/content/drive/MyDrive/A_christmas_carol_Dickens.txt', 'r') as f:
  scroodge_book = f.read()

In [36]:
len(re.findall(r"\n",scroodge_book))
filtered = re.sub(r"\n", "", scroodge_book)
n = 700
filtered = filtered[1704:]

In [41]:
print(len(re.findall(r"A CHRISTMAS CAROL", filtered)))
filtered = re.sub(r"A CHRISTMAS CAROL", r" ", filtered)


34


"  STAVE ONE MARLEY'S GHOST Marley was dead, to begin with. There is no doubt whatever about that. The register of his burial was signed by the clergyman, the clerk, the undertaker, and the chief mourner. Scrooge signed it. And Scrooge's name was good upon 'Change for anything he chose to put his hand to. Old Marley was as dead as a door- nail. Mind! I don't mean to say that I know, of my own knowledge, what there is particularly dead about a door nail. I might have been inclined, myself, to rega"

In [53]:
chapter_headers = ["STAVE ONE MARLEY'S GHOST", "STAVE TWO THE FIRST OF THE THREE SPIRITS", "STAVE THREE THE SECOND OF THE THREE SPIRITS"]
for i in range(3):
  filtered = re.sub(f"{chapter_headers[i]}", r"", filtered)
filtered[:500]

"   Marley was dead, to begin with. There is no doubt whatever about that. The register of his burial was signed by the clergyman, the clerk, the undertaker, and the chief mourner. Scrooge signed it. And Scrooge's name was good upon 'Change for anything he chose to put his hand to. Old Marley was as dead as a door- nail. Mind! I don't mean to say that I know, of my own knowledge, what there is particularly dead about a door nail. I might have been inclined, myself, to regard a coffin-nail as the "

In [85]:
sentence_list = re.findall(r"\..*?\.", filtered)

In [86]:
sentence_list[0:20]

['. There is no doubt whatever about that.',
 '. Scrooge signed it.',
 '. Old Marley was as dead as a door- nail.',
 '. I might have been inclined, myself, to regard a coffin-nail as the deadest piece of ironmongery in the trade.',
 '. You will therefore permit me to repeat, emphatically, that Marley was as dead as a door nail.',
 ". How could it be otherwise? Scrooge and he were partners for I don't know how many years.",
 '. And even Scrooge was not so dreadfully cut up by the sad event but that he was an excellent man of business on the very day of the funeral, and solemnized it with an undoubted bargain.',
 '. There is no doubt that Marley was dead.',
 ". If we were not perfectly convinced that Hamlet's Father died before the play began, there would be nothing more remarkable in his taking a stroll at night, in an easterly wind, upon his own ramparts, than there would be in any other middle-aged gentleman rashly turning out after dark in a breezy spot-say Saint Paul's Churchyard fo

In [153]:
sentence_list[3][1:].strip()


'I might have been inclined, myself, to regard a coffin-nail as the deadest piece of ironmongery in the trade.'

In [216]:
def clean_sentences(sentence_list):
  '''
  Add SOS - start of sentence and EOS end of sentence markers.
  Add EOSq for end of sentence question
  replace ; : with EOS
  replace ! with ,
  remove - i.e. treat every compound word as one unit
  remove all numbers
  remove \
  remove "
  '''
  cleaned_sentence_list = []
  for sentence in sentence_list:
    filtered = sentence[1:].strip()
    filtered = re.sub(r"[;:,]", r" ,", filtered)
    filtered = re.sub(r"\.", r" EOS", filtered)
    filtered = re.sub(r"\?", r" EOSq", filtered)
    filtered = re.sub(r"!", r",", filtered)
    filtered = re.sub(r"-", r"", filtered)
    filtered = re.sub(r"\d", r"", filtered)
    filtered = re.sub(r"\\", r"", filtered)
    filtered = re.sub(r"\"", r"", filtered)
    filtered = re.split(r" EOSq", filtered, concurrent=True)
    for i, string in enumerate(filtered):
      if i < len(filtered) - 1:
        string = f"SOS {string} EOS"
        cleaned_sentence_list.append(string.strip())
        continue
      cleaned_sentence_list.append(f"SOS {string.strip()}")
  return cleaned_sentence_list



In [217]:
cleaned_sentences = clean_sentences(sentence_list)

In [218]:
cleaned_sentences[350:400]

['SOS At last , however , he began to think as you or I would have thought at first , for it is always the person not in the predicament who knows what ought to have been done in it , and would unquestionably have done it too at last , I say , he began to think that the source and secret of this ghostly light might be in the adjoining room , fromwhence , on further tracing it , it seemed to shine EOS',
 "SOS The moment Scrooge's hand was on the lock , a strange voice called him by his name , and bade him enter EOS",
 'SOS It was his own room EOS',
 'SOS But it had undergone a surprising transformation EOS',
 "SOS The crisp leaves of holly , mistletoe , and ivy reflected back the light , as if so many little mirrors had been scattered there , and such a mighty blaze went roaring up the chimney , as that dull petrifac tion of a hearth had never known in Scrooge's time , or Marley's , or for many and many a winter season gone EOS",
 "SOS In easy state upon this couch there sat a jolly Gia

In [222]:
def generate_one_hot_mapping(sentence_list):
  '''
  '''
  words_to_int = {}
  int_to_words = {}
  words_to_int["EOS"] = 0
  int_to_words[0] = "EOS"
  size = 1
  for sentence in sentence_list:
    words = re.split(r"\s", sentence, concurrent=True)
    for word in words:
      if word in words_to_int.keys():
        continue
      else:
        words_to_int[word] = size
        int_to_words[size] = word
        size += 1
  return (words_to_int, int_to_words)


In [223]:
map_words, map_int = generate_one_hot_mapping(cleaned_sentences)

In [224]:
map_words

{'EOS': 0,
 'SOS': 1,
 'There': 2,
 'is': 3,
 'no': 4,
 'doubt': 5,
 'whatever': 6,
 'about': 7,
 'that': 8,
 'Scrooge': 9,
 'signed': 10,
 'it': 11,
 'Old': 12,
 'Marley': 13,
 'was': 14,
 'as': 15,
 'dead': 16,
 'a': 17,
 'door': 18,
 'nail': 19,
 'I': 20,
 'might': 21,
 'have': 22,
 'been': 23,
 'inclined': 24,
 ',': 25,
 'myself': 26,
 'to': 27,
 'regard': 28,
 'coffinnail': 29,
 'the': 30,
 'deadest': 31,
 'piece': 32,
 'of': 33,
 'ironmongery': 34,
 'in': 35,
 'trade': 36,
 'You': 37,
 'will': 38,
 'therefore': 39,
 'permit': 40,
 'me': 41,
 'repeat': 42,
 'emphatically': 43,
 'How': 44,
 'could': 45,
 'be': 46,
 'otherwise': 47,
 'and': 48,
 'he': 49,
 'were': 50,
 'partners': 51,
 'for': 52,
 "don't": 53,
 'know': 54,
 'how': 55,
 'many': 56,
 'years': 57,
 'And': 58,
 'even': 59,
 'not': 60,
 'so': 61,
 'dreadfully': 62,
 'cut': 63,
 'up': 64,
 'by': 65,
 'sad': 66,
 'event': 67,
 'but': 68,
 'an': 69,
 'excellent': 70,
 'man': 71,
 'business': 72,
 'on': 73,
 'very': 74,
 'da

In [191]:
def translate_to_num(sentence_list, dictionary):
  '''
  '''
  translated_sequences = []
  for sentence in sentence_list:
    words = re.split(r"\s", sentence, concurrent=True)
    sequence = []
    for word in words:
      sequence.append(dictionary[word])
    translated_sequences.append(sequence)
  return translated_sequences

In [225]:
string_sequences = translate_to_num(cleaned_sentences, map_words)

In [228]:
string_sequences[5]

[1, 44, 45, 11, 46, 47, 0]

In [208]:
map_int[0]

'EOS'

In [87]:
seq = string_sequences[8]


None ((<AddBackward0 object at 0x7c5975f11750>, 0),) ((<PowBackward0 object at 0x7c5975b1aa10>, 0), (None, 0)) 
((<DotBackward0 object at 0x7c5975f1ab30>, 0),) ((<AccumulateGrad object at 0x7c5975b45000>, 0), (<AccumulateGrad object at 0x7c5975b45360>, 0)) 
None None


/tmp/ipython-input-2088928086.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  print(dist3.grad, dist3.grad_fn.next_functions, dist2.grad_fn.next_functions, f"\n{dist.grad_fn.next_functions}", c.grad_fn.next_functions,


In [113]:
waveform, samplerate = torchaudio.load('/content/drive/MyDrive/Train_audio/audio1.wav')

In [114]:
class RNN(nn.RNNBase):

  def _init_(self, input_size, hidden_size, output_size):
    super().__init__()
    self.output_matrix = torch.rand((self.output_size, self.hidden_size), requires_grad=True)
    self.softmax = nn.Softmax(dim=0)
    self.loss = nn.CrossEntropyLoss()
    self.output_size = output_size
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.hidden_layer = nn.RNNCell(input_size, hidden_size)

  def forward(self, input_sequence):
    hidden_list = []
    h0 = torch.zeros((self.hidden_size))
    for input in input_sequence:
      h0 = self.hidden_layer(input, h0)
      hidden_list.append(h0)
    output_list = []
    for hidden in hidden_list:
      output = self.softmax(self.output_matrix @ hidden)
      output_list.append(output)
    return torch.vstack(output)


In [229]:
a = torch.tensor([1,2,3])
b = torch.tensor([4,5,6])
c = [a,b]
print(c)
d = torch.vstack(tuple(c))
d

[tensor([1, 2, 3]), tensor([4, 5, 6])]


tensor([[1, 2, 3],
        [4, 5, 6]])

In [233]:
a = [1,2,3]
a[:2]

[1, 2]

In [ ]:
def train(model, list_of_sequences, epochs):
  '''
  '''
  model.train()
  optim = torch.optim.Adam(model.parameters())
  for i in range(epochs):
    n = torch.randint(0,len(list_of_sequences) - 1)
    sequence = list_of_sequences[n]
    optim.zero_grad()
    output = model.forward(sequence[: len(sequence) - 1])
    loss = model.loss(output, sequence[1:])
    loss.backward()
    optim.step()

